In [39]:
import pymysql.cursors
import pandas as pd
from sklearn import preprocessing

# 训练

## 获取全量的竞彩比赛列表

In [26]:
connection = pymysql.connect(host='localhost', user='root', password='breadt@2019', db='breadt-football-ml', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
with connection.cursor() as cursor:
    sql = 'select * from `breadt_football_game_list`;'
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    train_game_list_df = pd.DataFrame(rows)
    
    connection.close()

## 获取全量的胜负彩比赛列表

In [28]:
connection = pymysql.connect(host='localhost', user='root', password='breadt@2019', db='breadt-football-ml', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
with connection.cursor() as cursor:
    sql = 'select * from `breadt_lottery_info`;'
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    train_lottery_game_list_df = pd.DataFrame(rows)
    
    connection.close()

## 合并竞彩比赛列表和胜负彩比赛列表

In [30]:
tmp = train_lottery_game_list_df.drop(['issue'], axis=1)
df = pd.concat([train_game_list_df, tmp])
df = df[['matchid', 'game', 'home_team', 'visit_team', 'gs', 'gd', 'gn', 'time', 'result', 'win_bet_return', 'draw_bet_return', 'lose_bet_return']]
df = df.drop_duplicates(subset=['matchid'])

## **设定训练范围** 并处理数据

In [49]:
match_group = ['澳超', '英超', '德甲', '德乙', '法甲', '西甲', '意甲', '日职', '英甲', '英冠', '苏超', '法乙', '葡超', '荷甲', '荷乙', '韩K联', '瑞典超', '挪超', '美职', '日乙', '俄超', '比甲', '瑞典甲', '法丙', '挪甲', '英乙', '苏冠', '巴甲', '智利甲', '墨超', '智利乙', '阿甲']
match_df = df[(df['game'].isin(match_group))]
match_df = match_df.dropna()

In [51]:
# 对team做encode 这个encoder后面预测的时候还会用到
teams = list(set(df['home_team'].values) | set(df['visit_team'].values))
team_encoder = preprocessing.LabelEncoder()
team_encoder.fit(teams)

LabelEncoder()

In [52]:
def encode_team(df):
    df['home_team_encoder'] = team_encoder.transform(df['home_team'])
    df['visit_team_encoder'] = team_encoder.transform(df['visit_team'])
    return df

In [53]:
# 比赛名称encode
games = list(set(match_df['game'].values))
game_encoder = preprocessing.LabelEncoder()
game_encoder.fit(games)

LabelEncoder()

In [54]:
def encode_game(df):
    df['game_encoder'] = game_encoder.transform(df['game'])
    return df

In [50]:
match_df['year'] = match_df.apply(lambda row: row.time.year, axis=1)
match_df['month'] = match_df.apply(lambda row: row.time.month, axis=1)
match_df['day'] = match_df.apply(lambda row: row.time.day, axis=1)
match_df['fix_result'] = match_df.apply(lambda row: int(row.result) if row.result < 3 else 2, axis=1)

In [55]:
match_df = encode_team(match_df)
match_df = encode_game(match_df)

In [56]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43311 entries, 1 to 12549
Data columns (total 19 columns):
matchid               43311 non-null int64
game                  43311 non-null object
home_team             43311 non-null object
visit_team            43311 non-null object
gs                    43311 non-null int64
gd                    43311 non-null int64
gn                    43311 non-null int64
time                  43311 non-null datetime64[ns]
result                43311 non-null int64
win_bet_return        43311 non-null float64
draw_bet_return       43311 non-null float64
lose_bet_return       43311 non-null float64
year                  43311 non-null int64
month                 43311 non-null int64
day                   43311 non-null int64
fix_result            43311 non-null int64
home_team_encoder     43311 non-null int64
visit_team_encoder    43311 non-null int64
game_encoder          43311 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(12), object

## 获取特征数据

In [101]:
connection = pymysql.connect(host='localhost', user='root', password='breadt@2019', db='breadt-football-ml', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
with connection.cursor() as cursor:
    sql = 'select * from `breadt_football_feature_info`;'
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    train_feature_df = pd.DataFrame(rows)
    
    connection.close()

## 处理特征数据

In [102]:
def get_count(col):
    arr = col.split(' ')
    if len(arr) == 2:
        return int(arr[1])
    
    return int(arr[0])

In [103]:
def get_count_rate(col, all_count):
    arr = col.split(' ')
    if len(arr) == 2:
        return int(arr[1]) / int(all_count)
    
    return int(arr[0])  / int(all_count)

In [104]:
def get_current_score(words, score, pos):
    t = score.replace(words, '')
    t = t.replace('胜', '')
    t = t.replace('平', '')
    t = t.replace('负', '')
    t = t.replace('分', '')
    
    arr = t.split('  ')
    
    if len(arr) == 3 and len(arr[2].split(' ')) == 2:
        arr = [arr[0], arr[1]] + arr[2].split(' ')
    
    if len(arr) != 4:
        return None
    
    if arr[pos] != '':
        return int(arr[pos])
    else:
        return None

In [105]:
h_cols = ['h_perf_win', 'h_perf_draw', 'h_perf_lose', 'h_host_win', 'h_host_draw', 'h_host_lose', 'h_battle_with_front_10_win', 'h_battle_with_front_10_draw', 'h_battle_with_front_10_lose', 'h_battle_with_end_10_win', 'h_battle_with_end_10_draw', 'h_battle_with_end_10_lose']
v_cols =['v_perf_win', 'v_perf_draw', 'v_perf_lose', 'v_host_win', 'v_host_draw', 'v_host_lose', 'v_battle_with_front_10_win', 'v_battle_with_front_10_draw', 'v_battle_with_front_10_lose', 'v_battle_with_end_10_win', 'v_battle_with_end_10_draw', 'v_battle_with_end_10_lose']

In [106]:
def take_count(df):
    cols = h_cols + v_cols
    for v in cols:
        df[v + '_count'] = df.apply(lambda row: get_count(row[v]), axis=1)
    
    df['h_count'] = df['h_perf_win_count'] + df['h_perf_draw_count'] + df['h_perf_lose_count']
    df['v_count'] = df['v_perf_win_count'] + df['v_perf_draw_count'] + df['v_perf_lose_count']
    
    return df

In [107]:
def take_rate(df):
    for v in h_cols:
        df[v + '_count_rate'] = df.apply(lambda row: get_count_rate(row[v], row.h_count), axis=1)

    for v in v_cols:
        df[v + '_count_rate'] = df.apply(lambda row: get_count_rate(row[v], row.v_count), axis=1)
        
    return df

In [108]:
def take_score(df):
    h_items = ['h_current_win_count', 'h_current_draw_count', 'h_current_lose_count']
    for k, v in enumerate(h_items):
        df[v] = df.apply(lambda row: get_current_score('主队：', row.h_score, k), axis=1)
    
    df['h_all_count'] = df['h_current_win_count'] + df['h_current_draw_count'] + df['h_current_lose_count']
    
    for v in h_items:
        df[v + '_rate'] = df[v] / df['h_all_count']
        
    df['h_current_score'] = df.apply(lambda row: get_current_score('主队：', row.h_score, 3), axis=1)
    
    ###########################################################################################
    v_items = ['v_current_win_count', 'v_current_draw_count', 'v_current_lose_count']
    for k, v in enumerate(v_items):
        df[v] = df.apply(lambda row: get_current_score('客队：', row.v_score, k), axis=1)
    
    df['v_all_count'] = df['v_current_win_count'] + df['v_current_draw_count'] + df['v_current_lose_count']
    
    for v in v_items:
        df[v + '_rate'] = df[v] / df['v_all_count']
        
    df['v_current_score'] = df.apply(lambda row: get_current_score('客队：', row.v_score, 3), axis=1)
    
    return df

In [109]:
def get_current_rank(rank):
    arr = rank.split('  ')
    
    if len(arr) == 2:
        rank_str = arr[1].replace('本赛季排名：', '')
        if rank_str != '':
            return int(rank_str)
    
    return None

In [110]:
def take_goal_info(prefix, df):
    target_cols = ['_perf_gs', '_perf_gd', '_perf_avg_gs', '_perf_avg_gd',
    '_host_gs', '_host_gd', '_host_avg_gs', '_host_avg_gd',
    '_r3_gs', '_r3_gd', '_r3_avg_gs', '_r3_avg_gd',
    '_perf_bet_high', '_perf_bet_low', '_host_bet_high',
    '_host_bet_low', '_host_0_1_goal', '_host_2_3_goal',
    '_host_ab_4_goal', '_host_0_goal', '_host_1_goal',
    '_host_2_goal', '_host_3_goal', '_host_4_goal', '_host_5_goal',
    '_host_6_goal', '_host_7_goal']

    for k in target_cols:
        df[prefix + k + '_rate'] = df[prefix + k] / df[prefix + '_count']
    
    return df

后面对于预测数据可以复用

In [111]:
train_feature_df = take_count(train_feature_df)
train_feature_df = take_rate(train_feature_df)
train_feature_df = take_score(train_feature_df)

train_feature_df['h_current_rank'] = train_feature_df.apply(lambda row: get_current_rank(row.h_rank), axis=1)
train_feature_df['v_current_rank'] = train_feature_df.apply(lambda row: get_current_rank(row.v_rank), axis=1)

train_feature_df = take_goal_info('h', train_feature_df)
train_feature_df = take_goal_info('v', train_feature_df)

## 合并训练数据

In [112]:
train_dataset_df = pd.merge(match_df, train_feature_df, on='matchid', how='left')
train_dataset_df = train_dataset_df.dropna()
train_dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39246 entries, 0 to 43315
Columns: 224 entries, matchid to v_host_7_goal_rate
dtypes: datetime64[ns](1), float64(180), int64(12), object(31)
memory usage: 67.4+ MB


## xgb训练

In [113]:
import xgboost as xgb

x_columns = [
    'year', 
    'month', 
    'win_bet_return', 'draw_bet_return','lose_bet_return', 'home_team_encoder', 'visit_team_encoder', 'game_encoder', 
    
    'h_perf_gs', 'h_perf_gd', 'h_perf_avg_gs', 'h_perf_avg_gd',
    'h_host_gs', 'h_host_gd', 'h_host_avg_gs', 'h_host_avg_gd',
    'h_r3_gs', 'h_r3_gd', 'h_r3_avg_gs', 'h_r3_avg_gd',
    'h_perf_bet_high', 'h_perf_bet_low', 'h_host_bet_high',
    'h_host_bet_low', 'h_host_0_1_goal', 'h_host_2_3_goal',
    'h_host_ab_4_goal', 'h_host_0_goal', 'h_host_1_goal',
    'h_host_2_goal', 'h_host_3_goal', 'h_host_4_goal', 'h_host_5_goal',
    'h_host_6_goal', 'h_host_7_goal', 
    
    'h_perf_gs_rate', 'h_perf_gd_rate', 'h_perf_avg_gs_rate', 'h_perf_avg_gd_rate',
    'h_host_gs_rate', 'h_host_gd_rate', 'h_host_avg_gs_rate', 'h_host_avg_gd_rate',
    'h_r3_gs_rate', 'h_r3_gd_rate', 'h_r3_avg_gs_rate', 'h_r3_avg_gd_rate',
    'h_perf_bet_high_rate', 'h_perf_bet_low_rate', 'h_host_bet_high_rate',
    'h_host_bet_low_rate', 'h_host_0_1_goal_rate', 'h_host_2_3_goal_rate',
    'h_host_ab_4_goal_rate', 'h_host_0_goal_rate', 'h_host_1_goal_rate',
    'h_host_2_goal_rate', 'h_host_3_goal_rate', 'h_host_4_goal_rate', 'h_host_5_goal_rate',
    'h_host_6_goal_rate', 'h_host_7_goal_rate', 
    
    'v_perf_gs', 'v_perf_gd',
    'v_perf_avg_gs', 'v_perf_avg_gd', 'v_host_gs', 'v_host_gd',
    'v_host_avg_gs', 'v_host_avg_gd', 'v_r3_gs', 'v_r3_gd',
    'v_r3_avg_gs', 'v_r3_avg_gd', 'v_perf_bet_high', 'v_perf_bet_low',
    'v_host_bet_high', 'v_host_bet_low', 'v_host_0_1_goal',
    'v_host_2_3_goal', 'v_host_ab_4_goal', 'v_host_0_goal',
    'v_host_1_goal', 'v_host_2_goal', 'v_host_3_goal', 'v_host_4_goal',
    'v_host_5_goal', 'v_host_6_goal', 'v_host_7_goal',
    
    'v_perf_gs_rate', 'v_perf_gd_rate',
    'v_perf_avg_gs_rate', 'v_perf_avg_gd_rate', 'v_host_gs_rate', 'v_host_gd_rate',
    'v_host_avg_gs_rate', 'v_host_avg_gd_rate', 'v_r3_gs_rate', 'v_r3_gd_rate',
    'v_r3_avg_gs_rate', 'v_r3_avg_gd_rate', 'v_perf_bet_high_rate', 'v_perf_bet_low_rate',
    'v_host_bet_high_rate', 'v_host_bet_low_rate', 'v_host_0_1_goal_rate',
    'v_host_2_3_goal_rate', 'v_host_ab_4_goal_rate', 'v_host_0_goal_rate',
    'v_host_1_goal_rate', 'v_host_2_goal_rate', 'v_host_3_goal_rate', 'v_host_4_goal_rate',
    'v_host_5_goal_rate', 'v_host_6_goal_rate', 'v_host_7_goal_rate',

    'h_perf_win_count', 'h_perf_draw_count', 'h_perf_lose_count',
    'h_host_win_count', 'h_host_draw_count', 'h_host_lose_count',
    'h_battle_with_front_10_win_count',
    'h_battle_with_front_10_draw_count',
    'h_battle_with_front_10_lose_count',
    'h_battle_with_end_10_win_count',
    'h_battle_with_end_10_draw_count',
    'h_battle_with_end_10_lose_count', 'v_perf_win_count',
    'v_perf_draw_count', 'v_perf_lose_count', 'v_host_win_count',
    'v_host_draw_count', 'v_host_lose_count',
    'v_battle_with_front_10_win_count',
    'v_battle_with_front_10_draw_count',
    'v_battle_with_front_10_lose_count',
    'v_battle_with_end_10_win_count',
    'v_battle_with_end_10_draw_count',
    'v_battle_with_end_10_lose_count',
    
    'h_perf_win_count_rate', 'h_perf_draw_count_rate', 'h_perf_lose_count_rate',
    'h_host_win_count_rate', 'h_host_draw_count_rate', 'h_host_lose_count_rate',
    'h_battle_with_front_10_win_count_rate',
    'h_battle_with_front_10_draw_count_rate',
    'h_battle_with_front_10_lose_count_rate',
    'h_battle_with_end_10_win_count_rate',
    'h_battle_with_end_10_draw_count_rate',
    'h_battle_with_end_10_lose_count_rate', 'v_perf_win_count_rate',
    'v_perf_draw_count_rate', 'v_perf_lose_count_rate', 'v_host_win_count_rate',
    'v_host_draw_count_rate', 'v_host_lose_count_rate',
    'v_battle_with_front_10_win_count_rate',
    'v_battle_with_front_10_draw_count_rate',
    'v_battle_with_front_10_lose_count_rate',
    'v_battle_with_end_10_win_count_rate',
    'v_battle_with_end_10_draw_count_rate',
    'v_battle_with_end_10_lose_count_rate',
    
    'h_count', 'h_all_count',
    'v_count', 'v_all_count',
    
    'h_current_win_count',
    'h_current_draw_count', 'h_current_lose_count', 
    'h_current_win_count_rate',
    'h_current_draw_count_rate', 'h_current_lose_count_rate', 
    
    'h_current_score',
    'v_current_win_count', 'v_current_draw_count',
    'v_current_lose_count', 
    
    'v_current_win_count_rate', 'v_current_draw_count_rate',
    'v_current_lose_count_rate',
    
    'v_current_score',
    'h_current_rank', 'v_current_rank']
    
params={
    'booster':'gbtree',
    # 这里手写数字是0-9，是一个多类的问题，因此采用了multisoft多分类器，
    'objective': 'multi:softmax', 
    'num_class':3, # 类数，与 multisoftmax 并用
    'gamma':0.1,  # 在树的叶子节点下一个分区的最小损失，越大算法模型越保守 。[0:]
    'max_depth':8, # 构建树的深度 [1:]
    #'lambda':450,  # L2 正则项权重
    'subsample':0.7, # 采样训练数据，设置为0.5，随机选择一般的数据实例 (0:1]
    'colsample_bytree':0.7, # 构建树树时的采样比率 (0:1]
    #'min_child_weight':12, # 节点的最少特征数
    'silent':1 ,
    
#     这部分需要调整
#     'eta': 0.05, # 如同学习率
    'eta': 0.01, # 如同学习率
    
    
    'seed':710,
    'nthread':4,# cpu 线程数,根据自己U的个数适当调整
}

train_dataset = train_dataset_df[train_dataset_df['year'] < 2019]
test_dataset = train_dataset_df[train_dataset_df['year'] == 2019]

xgtrain = xgb.DMatrix(train_dataset[x_columns], label=train_dataset['fix_result'])
xgtest = xgb.DMatrix(test_dataset[x_columns], label=test_dataset['fix_result'])

watchlist = [(xgtrain, 'train'),(xgtest, 'val')]

# num_rounds = 10000
# stop_rounds = 100

num_rounds = 10000
stop_rounds = 300


model = xgb.train(params, xgtrain, num_rounds, watchlist,early_stopping_rounds=stop_rounds)
print(model.best_iteration)

preds = model.predict(xgtest, ntree_limit=model.best_iteration)
preds

[1580]	train-merror:0.038153	val-merror:0.215477
[1581]	train-merror:0.038153	val-merror:0.215477
[1582]	train-merror:0.038018	val-merror:0.215941
[1583]	train-merror:0.037775	val-merror:0.215477
[1584]	train-merror:0.037694	val-merror:0.215941
[1585]	train-merror:0.037721	val-merror:0.215014
Stopping. Best iteration:
[1285]	train-merror:0.063605	val-merror:0.212234

1285


array([1., 2., 1., ..., 2., 1., 1.], dtype=float32)

In [114]:
from sklearn.metrics import accuracy_score
accuracy_score(test_dataset['fix_result'], preds)

0.7868396663577386

In [115]:
submission = pd.DataFrame({
    "matchid": test_dataset.matchid, 
    "result": test_dataset.fix_result,
    "pred": preds
})

submission.to_csv('../data/submission_2019.csv', index=False)

In [116]:
model.save_model('../model/xgb.model')

# 预测

In [19]:
# 确认预测轮次
issue = 19061

## 获取预测比赛列表

In [22]:
connection = pymysql.connect(host='localhost', user='root', password='breadt@2019', db='breadt-football-ml', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
with connection.cursor() as cursor:
    sql = 'select * from `breadt_lottery_predict_info` where issue = %d;' % (issue)
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    predict_game_df = pd.DataFrame(rows)
    
    connection.close()

In [117]:
predict_game_df = predict_game_df[(predict_game_df['game'].isin(match_group))]
predict_game_df = predict_game_df.dropna()

In [119]:
predict_game_df['year'] = predict_game_df.apply(lambda row: row.time.year, axis=1)
predict_game_df['month'] = predict_game_df.apply(lambda row: row.time.month, axis=1)
predict_game_df['day'] = predict_game_df.apply(lambda row: row.time.day, axis=1)

In [120]:
predict_game_df = encode_team(predict_game_df)
predict_game_df = encode_game(predict_game_df)

## 获取预测特征数据

In [122]:
connection = pymysql.connect(host='localhost', user='root', password='breadt@2019', db='breadt-football-ml', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
with connection.cursor() as cursor:
    sql = 'select * from `breadt_football_feature_info` where matchid in (select matchid from `breadt_lottery_predict_info` where issue = %d);' % (issue)
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    predict_feature_df = pd.DataFrame(rows)
    
    connection.close()

In [123]:
predict_feature_df = take_count(predict_feature_df)
predict_feature_df = take_rate(predict_feature_df)
predict_feature_df = take_score(predict_feature_df)

predict_feature_df['h_current_rank'] = predict_feature_df.apply(lambda row: get_current_rank(row.h_rank), axis=1)
predict_feature_df['v_current_rank'] = predict_feature_df.apply(lambda row: get_current_rank(row.v_rank), axis=1)

predict_feature_df = take_goal_info('h', predict_feature_df)
predict_feature_df = take_goal_info('v', predict_feature_df)

## 合并预测数据

In [124]:
predict_dataset_df = pd.merge(predict_game_df, predict_feature_df, on='matchid', how='left')
predict_dataset_df = predict_dataset_df.dropna()
predict_dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Columns: 223 entries, draw_bet_return to v_host_7_goal_rate
dtypes: datetime64[ns](1), float64(99), int64(90), object(33)
memory usage: 17.5+ KB


In [125]:
xgpredict = xgb.DMatrix(predict_dataset_df[x_columns])

official_preds = model.predict(xgpredict, ntree_limit=model.best_iteration)
official_preds

array([1., 1., 2., 2., 0., 0., 2., 0., 2., 2.], dtype=float32)

In [128]:
pd.DataFrame({
    "matchid": predict_dataset_df.matchid, 
    "home_team": predict_dataset_df.home_team, 
    "visit_team": predict_dataset_df.visit_team, 
    "game": predict_dataset_df.game, 
    "pred": official_preds
})

,matchid,home_team,visit_team,game,pred
0,2411095,米尔沃尔,布城,英冠,1.0
1,2411100,斯旺西,德比郡,英冠,1.0
2,2402946,蒙彼利埃,巴黎圣曼,法甲,2.0
3,2402948,雷恩,摩纳哥,法甲,2.0
4,2529128,科林蒂安,沙佩科,巴甲,0.0
5,2529126,巴西国际,弗拉门戈,巴甲,0.0
6,2529124,阿瓦伊,格雷米奥,巴甲,2.0
7,2529132,克鲁塞罗,塞阿拉,巴甲,0.0
8,2529130,达伽马,米竞技,巴甲,2.0
9,2529127,戈亚斯,圣保罗,巴甲,2.0
